In [1]:
!nvidia-smi

Sun Oct  1 20:58:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/NCKH/ViMMRC_model/'

/content/drive/MyDrive/NCKH/ViMMRC_model


# Library

https://github.com/mhardalov/exams-qa \
https://github.com/microsoft/MT-DNN \
https://github.com/jind11/MMM-MCQA

In [ ]:
#%pip -q install -e git+https://github.com/microsoft/MT-DNN.git#egg=mtdnn

In [4]:
%pip -q install torch -U
%pip -q install transformers -U
%pip -q install tqdm -U
# %pip -q install --upgrade  git+https://github.com/lanpa/tensorboardX.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00


Install apex package: https://stackoverflow.com/a/74561776

Query the version Ubuntu Colab is running on:\
`!lsb_release -a`

Get the current cuda version run:\
`!nvcc --version`

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# Models

In [6]:
#%pip install git+git@github.com:microsoft/mt-dnn.git@master#egg=mtdnn

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
# from mtdnn.common.dropout_wrapper import DropoutWrapper
# from mtdnn.common.optimizer import weight_norm as WN
# from mtdnn.common.similarity import FlatSimilarityWrapper, SelfAttnWrapper, SimilarityWrapper
# from mtdnn.common.activation_functions import activation


# class DualAttentionWrapper(nn.Module):

# class Classifier(nn.Module):


# # mtdnn.common.san.SANClassifier() super
# class SANClassifier(nn.Module):
#     """Implementation of Stochastic Answer Networks for Natural Language Inference, Xiaodong Liu, Kevin Duh and Jianfeng Gao
#     https://arxiv.org/abs/1804.07888
#     """

# class SANClassifier2(nn.Module):
#     """Implementation of Stochastic Answer Networks for Natural Language Inference, Xiaodong Liu, Kevin Duh and Jianfeng Gao
#     https://arxiv.org/abs/1804.07888
#     """

In [8]:
# MMM-MCQA from https://github.com/jind11/MMM-MCQA

#import copy
import math
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, NLLLoss
from transformers.modeling_utils import prune_linear_layer
from transformers.models.bert.modeling_bert import (
    BertPreTrainedModel,
    BertEmbeddings,
    BertSelfOutput,
    BertIntermediate,
    BertOutput,
    BertPooler,
    #BertModel,
)

BertLayerNorm = torch.nn.LayerNorm


# Utils

In [9]:
import sys
import csv
import glob
import json
import logging
import os
from typing import List

import numpy as np
from tqdm.auto import tqdm, trange
from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None, text_c=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence. Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.text_c = text_c
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, guid, input_ids, input_mask, segment_ids, label_id): #, premise_mask, hypothesis_mask):
        self.guid = guid
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        #self.premise_mask = premise_mask
        #self.hypothesis_mask = hypothesis_mask


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines

    @classmethod
    def _read_json(cls, input_file):
        """Reads a json file."""
        with open(input_file, "r", encoding='utf-8') as fpr:
            raw_list = json.load(fpr)
            return raw_list

    @classmethod
    def _read_jsonl(cls, input_file):
        """Reads a json file."""
        with open(input_file, "r", encoding='utf-8') as fpr:
            raw_list = list()
            for json_str in list(fpr):
                raw_list.append(json.loads(json_str))
            return raw_list


class VimmrcProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""
    def get_train_examples(self, data_dir):
        """See base class."""
        return self._read_samples(data_dir, "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._read_samples(data_dir, "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._read_samples(data_dir, "test")

    def get_labels(self):
        """See base class."""
        return ["A", "B", "C", "D"]

    def _read_samples(self, data_dir, set_type):
        """Creates examples for the training and dev sets."""
        # Init reader
        examples = []
        #example_id = 0
        # with open(filename, 'r', encoding='utf-8') as fpr:
        #     raw_list = json.load(fpr)
        raw_list = self._read_json(os.path.join(data_dir, set_type + ".json"))

        for data_raw in raw_list:
            # data_raw = json.load(fpr)
            article = data_raw['content']
            example_id = 0
            title = '_'.join(data_raw['files'].split('/')[-1].split('_')[:-1])
            for i in range(len(data_raw['questions'])):
                example_id += 1
                #truth = str(ord(data_raw['questions'][i]['answer']) - ord('A'))
                truth = data_raw['questions'][i]['answer']
                question = data_raw['questions'][i]['question']
                options = data_raw['questions'][i]['options']
                for k in range(len(options)):
                    guid = "%s-%s-%s-%s" % (set_type, title, example_id, k)
                    option = list(options[k].values())[0]
                    examples.append(
                        InputExample(guid=guid, text_a=article, text_b=option, label=truth, text_c=question))

        return examples


class VinliProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""
    def get_train_examples(self, data_dir):
        """See base class."""
        return self._read_samples(data_dir, "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._read_samples(data_dir, "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._read_samples(data_dir, "test")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _read_samples(self, data_dir, set_type):
        """Creates examples for the training and dev sets."""
        # Init reader
        examples = []
        raw_list = self._read_jsonl(os.path.join(data_dir, set_type + ".jsonl"))
        for data_line in raw_list:
            guid = "%s-%s" % (set_type, data_line['pairID'])
            text_a = data_line['sentence1']
            text_b = data_line['sentence2']
            label = data_line['gold_label']
            if label in self.get_labels():
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))

        return examples


class RaceProcessor(DataProcessor):
    """Processor for the RACE data set."""

    def get_train_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} train".format(data_dir))
        high = os.path.join(data_dir, "train_race.json")
        # middle = os.path.join(data_dir, "train/middle")
        high = self._read_txt(high)
        # middle = self._read_txt(middle)
        return self._create_examples(high, "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        high = os.path.join(data_dir, "dev_race.json")
        # middle = os.path.join(data_dir, "dev/middle")
        high = self._read_txt(high)
        # middle = self._read_txt(middle)
        return self._create_examples(high, "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} test".format(data_dir))
        high = os.path.join(data_dir, "test_race.json")
        # middle = os.path.join(data_dir, "test/middle")
        high = self._read_txt(high)
        # middle = self._read_txt(middle)
        return self._create_examples(high, "test")

    def get_labels(self):
        """See base class."""
        return ["A", "B", "C", "D"]

    def _read_txt(self, input_dir):
        lines = []
        # files = glob.glob(input_dir + "/*json")
        # for file in tqdm(files, desc="read files", disable = True):
        #     with open(file, "r", encoding="utf-8") as fin:
        #         data_raw = json.load(fin)
        #         data_raw["race_id"] = file
        #         lines.append(data_raw)
        with open(input_dir, "r", encoding="utf-8") as fin:
            data_raw = json.load(fin)
        for d in data_raw:
            d['race_id'] = d['files']
            lines.append(d)
        return lines

    # def _create_examples(self, lines, set_type):
    #     """Creates examples for the training and dev sets."""
    #     examples = []
    #     for (_, data_raw) in enumerate(lines):
    #         race_id = "%s-%s" % (set_type, data_raw["race_id"])
    #         article = data_raw["article"]
    #         for i in range(len(data_raw["answers"])):
    #             truth = str(ord(data_raw["answers"][i]) - ord("A"))
    #             # try:
    #             #     truth = str(ord(data_raw["answers"][i]) - ord("A"))
    #             # except Exception as e:
    #             #     print(data_raw["answers"][i])
    #             #     print("-----")
    #             #     print(data_raw["answers"])
    #             #     raise e
    #             question = data_raw["questions"][i]
    #             options = data_raw["options"][i]

    #             examples.append(
    #                 InputExample(
    #                     example_id=race_id,
    #                     question=question,
    #                     contexts=[
    #                         article,
    #                         article,
    #                         article,
    #                         article,
    #                     ],  # this is not efficient but convenient
    #                     endings=[options[0], options[1], options[2], options[3]],
    #                     label=truth,
    #                 )
    #             )
    #     return examples
    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (_, data_raw) in enumerate(lines):
            race_id = "%s-%s" % (set_type, data_raw["race_id"])
            article = data_raw["article"]
            for i in range(len(data_raw["answers"])):
                truth = data_raw["answers"][i]
                question = data_raw["questions"][i]
                options = data_raw["options"][i]
                for k in range(len(options)):
                    guid = "%s-%s" % (race_id, k)
                    examples.append(
                        InputExample(
                            guid=guid,
                            text_a=article,
                            text_b=options[k], #list(options[k].values())[0]
                            label=truth,
                            text_c=question
                        )
                    )
        return examples


class InfiniteDataLoader:
    def __init__(self, data_loader):
        self.data_loader = data_loader
        self.data_iter = iter(data_loader)

    def get_next(self):
        try:
            data = next(self.data_iter)
        except StopIteration:
            # StopIteration is thrown if dataset ends
            # reinitialize data loader
            self.data_iter = iter(self.data_loader)
            data = next(self.data_iter)
        return data


def convert_examples_to_features(
    examples: List[InputExample],
    label_list: List[str],
    max_seq_length: int,
    tokenizer,
    n_class,
    do_lower_case,
    output_mode,
    set_type,
    is_multi_choice=True
) -> List[InputFeatures]:
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label: i for i, label in enumerate(label_list)}

    if is_multi_choice:
        features = [[]]
    else:
        features = []

    feature_iterator = tqdm(examples, desc="Convert examples to features")
    for (ex_index, example) in enumerate(feature_iterator):
        feature_iterator.set_description("Convert %d of %d example to features" % (ex_index, len(examples)))

        tokens_a = tokenizer.tokenize(example.text_a.lower() if do_lower_case else example.text_a)  # dialogues

        tokens_b = [] # None
        tokens_c = [] # None

        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b.lower() if do_lower_case else example.text_b)  # answers

        if example.text_c:
            tokens_c = tokenizer.tokenize(example.text_c.lower() if do_lower_case else example.text_c)  # questions

        if tokens_c:
            _truncate_seq_tuple(tokens_a, tokens_b, tokens_c, max_seq_length - 4)
            tokens_b = tokens_c + ["[SEP]"] + tokens_b
        elif tokens_b:
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)]

        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = (len(tokens_a) + 2) * [0]
        #premise_mask = (len(tokens_a) + 2) * [1]
        #hypothesis_mask = (len(tokens_a) + 2) * [0]

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)
            #premise_mask += [1] * (len(tokens_c) + 1)
            #premise_mask += [0] * (len(tokens_b) - len(tokens_c))
            #hypothesis_mask += [0] * (len(tokens_c) + 1)
            #hypothesis_mask += [1] * (len(tokens_b) - len(tokens_c))

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        pad_length = max_seq_length - len(input_ids)
        input_ids += [0] * pad_length
        input_mask += [0] * pad_length
        segment_ids += [0] * pad_length
        #premise_mask += [0] * pad_length
        #hypothesis_mask += [0] * pad_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        #assert len(premise_mask) == max_seq_length
        #assert len(hypothesis_mask) == max_seq_length

        if output_mode in ["classification", "multi-choice"]:
            label_id = label_map[example.label]
        elif output_mode == "regression":
            label_id = float(example.label)
        else:
            raise KeyError(output_mode)

        if is_multi_choice:
            features[-1].append(
                InputFeatures(
                    guid=example.guid,
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    label_id=label_id,
                    #premise_mask = premise_mask,
                    #hypothesis_mask = hypothesis_mask
                    ))
            if len(features[-1]) == n_class:
                features.append([])
        else:
            features.append(
                InputFeatures(
                    guid=example.guid,
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    label_id=label_id,
                    #premise_mask = premise_mask,
                    #hypothesis_mask = hypothesis_mask
                    ))

        ## display some example
        if set_type == 'train' and ex_index < 4:
            logger.info("*** Example ***")
            logger.info(f"guid: {example.guid}")
            logger.info(f"tokens: {' '.join(tokens)}")
            logger.info(f"input_ids: {' '.join(map(str, input_ids))}")
            logger.info(f"input_mask: {' '.join(map(str, input_mask))}")
            logger.info(f"segment_ids: {' '.join(map(str, segment_ids))}")
            #logger.info(f"premis_mask: {' '.join(map(str, premise_mask))}")
            #logger.info(f"hypoth_mask: {' '.join(map(str, hypothesis_mask))}")
            try:
                logger.info(f"label: {example.label}")
            except:
                pass

    if is_multi_choice:
        if len(features[-1]) == 0:
            features = features[:-1]

    return features


def convert_features_to_tensors(features, output_mode, is_multi_choice=True):

    input_ids = []
    input_mask = []
    segment_ids = []
    label_id = []
    #premise_mask = []
    #hypothesis_mask = []

    if is_multi_choice:
        n_class = len(features[0])
        for f in features:
            input_ids.append([])
            input_mask.append([])
            segment_ids.append([])
            #premise_mask.append([])
            #hypothesis_mask.append([])
            for i in range(n_class):
                input_ids[-1].append(f[i].input_ids)
                input_mask[-1].append(f[i].input_mask)
                segment_ids[-1].append(f[i].segment_ids)
                #premise_mask[-1].append(f[i].premise_mask)
                #hypothesis_mask[-1].append(f[i].hypothesis_mask)

            label_id.append([f[0].label_id])
    else:
        for f in features:
            input_ids.append(f.input_ids)
            input_mask.append(f.input_mask)
            segment_ids.append(f.segment_ids)
            label_id.append(f.label_id)
            #premise_mask.append(f.premise_mask)
            #hypothesis_mask.append(f.hypothesis_mask)

    all_input_ids = torch.tensor(input_ids, dtype=torch.long)
    all_input_mask = torch.tensor(input_mask, dtype=torch.long)
    all_segment_ids = torch.tensor(segment_ids, dtype=torch.long)
    #all_premise_mask = torch.tensor(premise_mask, dtype=torch.bool)
    #all_hypothesis_mask = torch.tensor(hypothesis_mask, dtype=torch.bool)

    if output_mode in ["classification", "multi-choice"]:
        all_label_ids = torch.tensor(label_id, dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor(label_id, dtype=torch.float)

    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
                         #all_premise_mask, all_hypothesis_mask,
                         #all_label_ids)

    return data


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def _truncate_seq_tuple(tokens_a, tokens_b, tokens_c, max_length):
    """Truncates a sequence tuple in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b) + len(tokens_c)
        if total_length <= max_length:
            break
        if len(tokens_a) >= len(tokens_b) and len(tokens_a) >= len(tokens_c):
            tokens_a.pop()
        elif len(tokens_b) >= len(tokens_a) and len(tokens_b) >= len(tokens_c):
            tokens_b.pop()
        else:
            tokens_c.pop()


processors = {
    "vimmrc": VimmrcProcessor,
    "vinli": VinliProcessor,
    "vimmrc_race": RaceProcessor,
}

output_modes = {
    "vimmrc": 'multi-choice',
    "vimmrc_race": 'multi-choice',
    "vinli": "classification",
}

GLUE_TASKS_NUM_LABELS = { "vimmrc": 4, "vimmrc_race": 4, "vinli": 3 }

MAX_SEQ_LENGTHS = { "vimmrc": 512, "vimmrc_race": 512, "vinli": 128 }

# Module

In [10]:
import argparse
import glob
import json
import logging
import os
import random

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
#from torch.optim import AdamW
from tqdm.auto import tqdm, trange
from transformers import (
    WEIGHTS_NAME, CONFIG_NAME,
    AdamW,
    BertConfig,
    BertForMultipleChoice,
    BertTokenizer,
    RobertaConfig,
    RobertaForMultipleChoice,
    RobertaTokenizer,
    XLMRobertaConfig,
    XLMRobertaForMultipleChoice,
    XLMRobertaTokenizer,
    XLNetConfig,
    XLNetForMultipleChoice,
    XLNetTokenizer,
    get_linear_schedule_with_warmup,
)

from transformers import BERT_PRETRAINED_CONFIG_ARCHIVE_MAP, DISTILBERT_PRETRAINED_CONFIG_ARCHIVE_MAP, XLM_ROBERTA_PRETRAINED_CONFIG_ARCHIVE_MAP
from transformers import DistilBertConfig, DistilBertForMultipleChoice, DistilBertTokenizer

# try:
#     from torch.utils.tensorboard import SummaryWriter
# except ImportError:
#     from tensorboardX import SummaryWriter

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s', datefmt='%m/%d/%Y %H:%M:%S', level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

ALL_MODELS = tuple(DISTILBERT_PRETRAINED_CONFIG_ARCHIVE_MAP.keys()) + tuple(BERT_PRETRAINED_CONFIG_ARCHIVE_MAP.keys()) + tuple(XLM_ROBERTA_PRETRAINED_CONFIG_ARCHIVE_MAP.keys())

MODEL_CLASSES = {
    "bert": (BertConfig, BertForMultipleChoice, BertTokenizer),
    #"bert-man": (BertConfig, BertForMultipleChoice_SAN, BertTokenizer),
    #"roberta": (RobertaConfig, RobertaForMultipleChoice, RobertaTokenizer),
    #"xlm-roberta": (XLMRobertaConfig, XLMRobertaForMultipleChoice, XLMRobertaTokenizer)
}

In [11]:
def select_field(features, field):
    return [[choice[field] for choice in feature.choices_features] for feature in features]


def simple_accuracy(preds, labels):
    return (preds == labels).mean()


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def load_and_cache_examples(args, task, tokenizer, set_type='train'):
    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    output_mode = output_modes[task]
    is_multi_choice = True if output_mode == 'multi-choice' else False
    processor = processors[task]()
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(
        args.data_dir,
        'cached_{}_{}_{}_{}'.format(
            set_type,
            list(filter(None, args.model_name_or_path.split('/'))).pop(),
            str(MAX_SEQ_LENGTHS[task]),
            str(task)
        )
    )
    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        if set_type == 'train':
            examples = processor.get_train_examples(args.data_dir)
        elif set_type == 'dev':
            examples = processor.get_dev_examples(args.data_dir)
        else:
            examples = processor.get_test_examples(args.data_dir)
        logger.info("Training number: %s", str(len(examples)))
        features = convert_examples_to_features(
            examples,
            label_list,
            MAX_SEQ_LENGTHS[task],
            tokenizer,
            len(label_list),
            output_mode=output_mode,
            set_type=set_type,
            do_lower_case=args.do_lower_case,
            is_multi_choice=is_multi_choice
        )
        if args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)

    # Convert to Tensors and build dataset
    dataset = convert_features_to_tensors(
        features, output_mode, is_multi_choice=is_multi_choice
    )

    if set_type == 'dev' or set_type == 'test':
        all_example_ids = [x.guid for feature_set in features for x in feature_set]#[x.example_id for x in features]
        return dataset, all_example_ids
    else:
        return dataset

In [12]:
def train(args, train_datasets, model, tokenizer):
    """ Train the model """
    # if args.local_rank in [-1, 0]:
    #     tb_writer = SummaryWriter()
    #     pass

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_iters = []
    tr_batches = []
    #for idx, train_dataset in enumerate(train_datasets):
    train_sampler = RandomSampler(train_datasets) if args.local_rank == -1 else DistributedSampler(train_datasets)
    train_dataloader = DataLoader(train_datasets, sampler=train_sampler, batch_size=args.train_batch_size)
    train_iters.append(InfiniteDataLoader(train_dataloader))
    tr_batches.append(len(train_dataloader))

    ## set sampling proportion
    total_n_tr_batches = sum(tr_batches)
    sampling_prob = [float(n_batches) / total_n_tr_batches for n_batches in tr_batches]
    t_total = total_n_tr_batches // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
         'weight_decay': 0.0}
    ]

    # optimizer = BertAdam(
    #     optimizer_grouped_parameters,
    #     lr=args.learning_rate,
    #     warmup=args.warmup_proportion,
    #     max_grad_norm=args.max_grad_norm, t_total=t_total)

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, correct_bias=False, no_deprecation_warning=True) # To reproduce BertAdam specific behavior set correct_bias=False
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(args.warmup_proportion * t_total),
        num_training_steps=t_total,
    ) # PyTorch scheduler

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_datasets))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %s", args.per_gpu_train_batch_size)
    logger.info(
        " Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss, tmp_loss = 0.0, 0.0, 0.0
    best_dev_acc = 0.0
    best_steps = 0
    nb_tr_examples = 0
    model.zero_grad()
    train_iterator = tqdm(range(int(args.num_train_epochs)), desc="Epoch") #, disable=args.local_rank not in [-1, 0])
    set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
    for epoch, _ in enumerate(train_iterator):
        epoch_iterator = tqdm(range(total_n_tr_batches), desc="Iteration") #, disable=args.local_rank not in [-1, 0])
        for step, _ in enumerate(epoch_iterator):
            epoch_iterator.set_description(
                "train loss: {}".format(tr_loss / nb_tr_examples if nb_tr_examples else tr_loss)
            )
            model.train()
            # select task id
            task_id = np.argmax(np.random.multinomial(1, sampling_prob))
            batch = train_iters[task_id].get_next()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      #'premise_mask':   batch[3],
                      #'hyp_mask':       batch[4],
                      'labels':         batch[3].view(-1),
                      #'task_id':        task_id
                      }
            outputs = model(**inputs)  # model outputs are always tuple in transformers (see doc)
            loss = outputs[0]

            if args.n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()
            nb_tr_examples += inputs['input_ids'].size(0)
            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1
                #tb_writer.add_scalar("train_{}".format("loss_"), tr_loss - tmp_loss, global_step)
                tmp_loss = tr_loss

        # Save model checkpoint
        # if args.do_epoch_checkpoint:
        epoch_output_dir = os.path.join(args.output_dir, 'epoch_{}'.format(epoch+1))
        os.makedirs(epoch_output_dir, exist_ok=True)
        output_model_file = os.path.join(epoch_output_dir, WEIGHTS_NAME)
        output_config_file = os.path.join(epoch_output_dir, CONFIG_NAME)
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        torch.save(model_to_save.state_dict(), output_model_file)
        model_to_save.config.to_json_file(output_config_file)
        tokenizer.save_vocabulary(epoch_output_dir)

        # evaluate(args, model, tokenizer, epoch=epoch, is_test=False)
        # evaluate(args, model, tokenizer, epoch=epoch, is_test=True)
    # if args.local_rank in [-1, 0]:
    #     tb_writer.close()

    return global_step, tr_loss / global_step, best_steps

In [13]:
def evaluate(args, model, tokenizer, checkpoint='', is_test=False, error=False):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_task_names = args.task_name
    eval_output_dir = args.output_dir
    if checkpoint:
        eval_output_dir=checkpoint

    set_type = 'test' if is_test else 'dev'
    results = {}
    #for task_id, eval_task in enumerate(eval_task_names):
    #for eval_task, eval_output_dir in zip(eval_task_names, eval_outputs_dirs):
    eval_dataset, all_example_ids = load_and_cache_examples(args, eval_task_names, tokenizer, set_type)

    if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    logger.info("***** Running evaluation for {} on {} for {} *****".format(eval_task_names, set_type, checkpoint))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    logits_all = None
    out_label_ids = None
    preds_softmax = None
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      #'premise_mask':   batch[3],
                      #'hyp_mask':       batch[4],
                      'labels':         batch[3].view(-1),
                      #'task_id':        0
                      }
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]
            # input_ids, input_mask, segment_ids, label_ids = batch
            # tmp_eval_loss, logits = model(input_ids, segment_ids, input_mask, label_ids, task_id=task_id)
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if logits_all is None:
            logits_all = logits.detach().cpu().numpy()
            preds_softmax = torch.softmax(logits, -1).detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            logits_all = np.append(logits_all, logits.detach().cpu().numpy(), axis=0)
            preds_softmax = np.append(preds_softmax, torch.softmax(logits, -1).detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    preds_softmax = [[round(x.tolist(), 4) for x in p] for p in preds_softmax]
    example_ids = [id[:-2] for id in all_example_ids[::4]]
    per_id_pred = dict(zip(example_ids, preds_softmax))
    output_mode = output_modes[eval_task_names]

    preds = np.argmax(logits_all, axis=1)
    acc = simple_accuracy(preds, out_label_ids.reshape(-1))
    #result = compute_metrics(eval_task, preds, out_label_ids.reshape(-1))
    result = {"eval_acc": acc, "eval_loss": eval_loss}
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results_{}_{}.txt".format(eval_task_names, set_type))
    with open(output_eval_file, "a") as writer:
        logger.info("***** Eval results for {} on {} for {} *****".format(eval_task_names, set_type, checkpoint))
        writer.write("***** Eval results for {} *****\n".format(checkpoint))
        writer.write(
            "total batch size=%d\n"
            % (
                args.per_gpu_train_batch_size
                * args.gradient_accumulation_steps
                * (torch.distributed.get_world_size() if args.local_rank != -1 else 1)
            )
        )
        writer.write("train num epochs=%d\n" % args.num_train_epochs)
        writer.write("max seq length  =%d\n" % MAX_SEQ_LENGTHS[eval_task_names])#args.max_seq_length)
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))
        logger.info("\n")

    # import pdb
    # pdb.set_trace()

    # Write pred
    output_pred_file = os.path.join(eval_output_dir, "predictions_" + str(is_test).lower() + "_eval_results.json")
    with open(output_pred_file, "w") as writer:
        json.dump(per_id_pred, writer, indent=4)
        logger.info("Saved {0}".format(output_pred_file))

    # Write pred with label
    processor = processors[args.task_name]()
    label_list = processor.get_labels()#['A','B','C','D']
    idx2label = {i: label for i, label in enumerate(label_list)}
    output_pred_file = os.path.join(eval_output_dir, "predictions_" + str(is_test).lower() + "_eval_results_label.json")
    #output_pred_file = '/content/drive/MyDrive/pred_test_labels.json'
    with open(output_pred_file, "w") as writer:
        json.dump({'pred': [idx2label[id] for id in preds.tolist()]}, writer, indent=4)
        logger.info("Saved {0}".format(output_pred_file))

    # Get error idx
    if error:
        correct_idx = np.argwhere(preds == out_label_ids).tolist()
        wrong_idx = np.argwhere(preds != out_label_ids).tolist()
        wrong_idx_dict = {
            'correct': correct_idx, 'wrong': wrong_idx,
            'preds': preds.tolist(), 'logits': logits_all.tolist(),
            'labels': out_label_ids.tolist()
        }
        output_error_file = os.path.join(eval_output_dir,"error_idx_{}_{}.json".format(eval_task_names, set_type))
        with open(output_error_file, "w") as writer:
            json.dump(wrong_idx_dict, writer)
            logger.info("Saved {0}".format(output_error_file))

    return results

# Main

In [24]:
PRJ_DIR = '/content/drive/MyDrive/NCKH/ViMMRC_model'
OUTPUT_DIR = PRJ_DIR + '/models/mbert_nli_base_v1'

In [25]:
def main(args):

    # Setup distant debugging if needed
    # if args.server_ip and args.server_port:
    #     # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
    #     import ptvsd

    #     print("Waiting for debugger attach")
    #     ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
    #     ptvsd.wait_for_attach()

    # Setup CUDA, GPU & distributed training
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        torch.distributed.init_process_group(backend="nccl")
        args.n_gpu = 1
    logger.info("device: %s, n_gpu: %d, distributed training %r", device, args.n_gpu, bool(args.local_rank != -1))
    args.device = device

    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
            args.gradient_accumulation_steps))

    if os.path.exists(args.output_dir) and os.listdir(args.output_dir):
        if args.do_train:
            print("Output directory ({}) already exists and is not empty.".format(args.output_dir))
    else:
        os.makedirs(args.output_dir, exist_ok=True)

    # Set seed
    set_seed(args)

    # Prepare GLUE task
    args.task_name = args.task_name.lower()
    if args.task_name not in processors:
        raise ValueError("Task not found: %s" % (args.task_name))
    processor = processors[args.task_name]()
    label_list = processor.get_labels()
    num_labels = len(label_list)

    # Load pretrained model and tokenizer
    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    args.model_type = args.model_type.lower()
    config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    config = config_class.from_pretrained(
        args.config_name if args.config_name else args.model_name_or_path,
        num_labels=num_labels,
        early_stopping = True,
        finetuning_task=args.task_name,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    tokenizer = tokenizer_class.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
        do_lower_case=args.do_lower_case,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    model = model_class.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
        cache_dir=args.cache_dir if args.cache_dir else None,
        ignore_mismatched_sizes=True
    )

    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    options_print = ""
    logging.info("Arg Options:")
    for arg in vars(args):
        options_print += "opt: %s=%s\r\n" % (arg, getattr(args, arg))
    logging.info(options_print)

    model.to(args.device)

    def get_model_base_obj(model, model_type):
        if model_type == "bert":
            return model.bert
        elif model_type == "xlm-roberta" or model_type == "roberta":
            return model.roberta
        elif model_type == "distilbert":
            return model.distilbert
        else:
            raise ValueError("model_type='{0}' is not supported!")

    if args.freeze_embeddings:
        for param in list(get_model_base_obj(model, args.model_type).embeddings.parameters()):
            param.requires_grad = False
        logger.info("Froze Embedding Layer")

    # freeze_layers is a string "1,2,3" representing layer number
    if args.freeze_layers:
        layer_indexes = [int(x) for x in args.freeze_layers]
        for layer_idx in layer_indexes:
            for param in list(
                get_model_base_obj(model, args.model_type).encoder.layer[layer_idx].parameters()
            ):
                param.requires_grad = False
            logger.info("Froze Layer: %s", layer_idx)

    logger.info("Training/evaluation parameters %s", args)
    best_steps = 0

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, args.task_name, tokenizer, set_type='train')
        global_step, tr_loss, best_steps = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
    if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Create output directory if needed
        if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(args.output_dir)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = model_class.from_pretrained(args.output_dir)
        tokenizer = tokenizer_class.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        if not args.do_train:
            args.output_dir = args.model_name_or_path
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c)
                for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("/")[-1] if len(checkpoints) > 1 else ""
            model = model_class.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, global_step)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    if args.do_test and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c)
                for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("/")[-1]
            model = model_class.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, global_step, is_test=True)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)
    # if best_steps:
    #     logger.info("best steps of eval acc is the following checkpoints: %s", best_steps)
    return results


## Test

In [ ]:
parser = argparse.ArgumentParser()

if True:
    # Required parameters
    parser.add_argument(
        "--data_dir", default="{}/dataset/ViMMRC_RACE_v1".format(PRJ_DIR), type=str,
        help="The input data dir. Should contain the .tsv files (or other data files) for the task.",
    )

    parser.add_argument(
        "--model_type", default="bert", type=str,
        help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--model_name_or_path", default="{}/models/mbert_nli_base".format(PRJ_DIR), type=str,
        help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(ALL_MODELS),
    )

    parser.add_argument(
        "--task_name", default="vimmrc_race", type=str,
        help="The name of the task to train selected in the list: " + ", ".join(processors.keys()),
    )
    # parser.add_argument(
    #     "--para_type", default="per_choice", type=str,
    #     choices=["per_choice", "concat_choices", "ignore"],
    #     help="Paragraph building strategy for ARC (default: %(default)s)",
    # )
    parser.add_argument(
        "--output_predictions", default=True, type=bool, help="Whether to export the predictions from the eval step.",
    )
    parser.add_argument(
        "--output_dir", default=OUTPUT_DIR, type=str, help="The output directory where the model predictions and checkpoints will be written.",
    )
    parser.add_argument("--freeze_embeddings", default=False, action="store_true", help="Whether to freeze the embeeding layer.",)
    parser.add_argument("--freeze_layers", nargs="*", help="Whether to freeze the embeeding layer.",)

    # Other parameters
    parser.add_argument("--tb_log_dir", default="", type=str, help="Tensorboard log dir for the current experiment")
    parser.add_argument("--config_name", default="", type=str, help="Pretrained config name or path if not the same as model_name")
    parser.add_argument("--tokenizer_name", default="", type=str, help="Pretrained tokenizer name or path if not the same as model_name")
    parser.add_argument(
        "--cache_dir", default="{}/models/cached".format(PRJ_DIR), type=str, help="Where do you want to store the pre-trained models downloaded from s3",
    )
    parser.add_argument(
        "--max_seq_length", default=MAX_SEQ_LENGTHS['vimmrc'], type=int,
        help="The maximum total input sequence length after tokenization. Sequences longer than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument("--do_train", default = False, action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", default = False, action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument("--do_test", default = True, action="store_true", help="Whether to run test on the test set")
    parser.add_argument(
        "--evaluate_during_training", action="store_true", default = True, help="Run evaluation during training at each logging step.",
    )
    parser.add_argument(
        "--do_lower_case", action="store_true", default = False, help="Set this flag if you are using an uncased model.",
    )

    parser.add_argument("--per_gpu_train_batch_size", default=4, type=int, help="Batch size per GPU/CPU for training.",)
    parser.add_argument("--per_gpu_eval_batch_size", default=4, type=int, help="Batch size per GPU/CPU for evaluation.",)
    parser.add_argument(
        "--gradient_accumulation_steps", type=int, default=8, help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--learning_rate", default=3e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.01, type=float, help="Weight decay if we apply some.")
    # parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument("--num_train_epochs", default=10, type=float, help="Total number of training epochs to perform.")
    parser.add_argument("--max_steps", default=-1, type=int, help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
    parser.add_argument("--warmup_proportion", default=0.1, type=float, help="Linear warmup over warmup_proportion.")

    # parser.add_argument("--logging_steps", type=int, default=100, help="Log every X updates steps.")
    # parser.add_argument("--save_steps", type=int, default=500, help="Save checkpoint every X updates steps.")
    parser.add_argument(
        "--eval_all_checkpoints", default=True, action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number",
    )
    parser.add_argument("--no_cuda", default=False, action="store_true", help="Avoid using CUDA when available")
    # parser.add_argument(
    #     "--overwrite_output_dir", default = True, action="store_true", help="Overwrite the content of the output directory",
    # )
    # parser.add_argument(
    #     "--overwrite_cache", default = True, action="store_true", help="Overwrite the cached training and evaluation sets",
    # )
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")

    # parser.add_argument(
    #     "--fp16", action="store_true", help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    # )
    # parser.add_argument(
    #     "--fp16_opt_level", type=str, default="O1",
    #     help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
    #     "See details at https://nvidia.github.io/apex/amp.html",
    # )
    parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
    # parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    # parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")

    parser.add_argument('-f')

args = parser.parse_args()
vars(args)
# options_print = ""
# logger.info("Arg Options - input:")
# for arg in vars(args):
#     options_print += "opt: %s=%s\r\n" % (arg, getattr(args, arg))
# logger.info(options_print)

{'data_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/dataset/ViMMRC_RACE_v1',
 'model_type': 'bert',
 'model_name_or_path': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/vibert_nli_base',
 'task_name': 'vimmrc_race',
 'output_predictions': True,
 'output_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/vibert_nli_base_v1',
 'freeze_embeddings': False,
 'freeze_layers': None,
 'tb_log_dir': '',
 'config_name': '',
 'tokenizer_name': '',
 'cache_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/cached',
 'max_seq_length': 512,
 'do_train': False,
 'do_eval': False,
 'do_test': True,
 'evaluate_during_training': True,
 'do_lower_case': False,
 'per_gpu_train_batch_size': 4,
 'per_gpu_eval_batch_size': 4,
 'gradient_accumulation_steps': 8,
 'learning_rate': 3e-05,
 'weight_decay': 0.01,
 'max_grad_norm': 1.0,
 'num_train_epochs': 10,
 'max_steps': -1,
 'warmup_proportion': 0.1,
 'eval_all_checkpoints': True,
 'no_cuda': False,
 'seed': 42,
 'local_rank': -1,
 'f': '/root/.l

In [ ]:
if __name__ == "__main__":
    main(args)

INFO:__main__:device: cuda, n_gpu: 1, distributed training False
INFO:__main__:Training/evaluation parameters Namespace(data_dir='/content/drive/MyDrive/NCKH/ViMMRC_model/dataset/ViMMRC_RACE_v1', model_type='bert', model_name_or_path='/content/drive/MyDrive/NCKH/ViMMRC_model/models/vibert_nli_base', task_name='vimmrc_race', output_predictions=True, output_dir='/content/drive/MyDrive/NCKH/ViMMRC_model/models/vibert_nli_base_v1', freeze_embeddings=False, freeze_layers=None, tb_log_dir='', config_name='', tokenizer_name='', cache_dir='/content/drive/MyDrive/NCKH/ViMMRC_model/models/cached', max_seq_length=512, do_train=False, do_eval=False, do_test=True, evaluate_during_training=True, do_lower_case=False, per_gpu_train_batch_size=4, per_gpu_eval_batch_size=4, gradient_accumulation_steps=8, learning_rate=3e-05, weight_decay=0.01, max_grad_norm=1.0, num_train_epochs=10, max_steps=-1, warmup_proportion=0.1, eval_all_checkpoints=True, no_cuda=False, seed=42, local_rank=-1, f='/root/.local/sha

### epoch x test

In [17]:
parser = argparse.ArgumentParser()

if True:
    # Required parameters
    parser.add_argument(
        "--data_dir", default="{}/dataset/ViMMRC_RACE_v1".format(PRJ_DIR), type=str,
        help="The input data dir. Should contain the .tsv files (or other data files) for the task.",
    )

    parser.add_argument(
        "--model_type", default="bert", type=str,
        help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--model_name_or_path", default="{}/models/mbert_nli".format(PRJ_DIR), type=str,
        help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(ALL_MODELS),
    )

    parser.add_argument(
        "--task_name", default="vimmrc_race", type=str,
        help="The name of the task to train selected in the list: " + ", ".join(processors.keys()),
    )
    # parser.add_argument(
    #     "--para_type", default="per_choice", type=str,
    #     choices=["per_choice", "concat_choices", "ignore"],
    #     help="Paragraph building strategy for ARC (default: %(default)s)",
    # )
    parser.add_argument(
        "--output_predictions", default=True, type=bool, help="Whether to export the predictions from the eval step.",
    )
    parser.add_argument(
        "--output_dir", default=PRJ_DIR + '/models/mbert_nli_base'+'/epoch_8', type=str, help="The output directory where the model predictions and checkpoints will be written.",
    )
    parser.add_argument("--freeze_embeddings", default=False, action="store_true", help="Whether to freeze the embeeding layer.",)
    parser.add_argument("--freeze_layers", nargs="*", help="Whether to freeze the embeeding layer.",)

    # Other parameters
    parser.add_argument("--tb_log_dir", default="", type=str, help="Tensorboard log dir for the current experiment")
    parser.add_argument("--config_name", default="", type=str, help="Pretrained config name or path if not the same as model_name")
    parser.add_argument("--tokenizer_name", default="", type=str, help="Pretrained tokenizer name or path if not the same as model_name")
    parser.add_argument(
        "--cache_dir", default="{}/models/cached".format(PRJ_DIR), type=str, help="Where do you want to store the pre-trained models downloaded from s3",
    )
    parser.add_argument(
        "--max_seq_length", default=MAX_SEQ_LENGTHS['vimmrc'], type=int,
        help="The maximum total input sequence length after tokenization. Sequences longer than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument("--do_train", default = False, action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", default = False, action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument("--do_test", default = True, action="store_true", help="Whether to run test on the test set")
    parser.add_argument(
        "--evaluate_during_training", action="store_true", default = True, help="Run evaluation during training at each logging step.",
    )
    parser.add_argument(
        "--do_lower_case", action="store_true", default = False, help="Set this flag if you are using an uncased model.",
    )

    parser.add_argument("--per_gpu_train_batch_size", default=4, type=int, help="Batch size per GPU/CPU for training.",)
    parser.add_argument("--per_gpu_eval_batch_size", default=4, type=int, help="Batch size per GPU/CPU for evaluation.",)
    parser.add_argument(
        "--gradient_accumulation_steps", type=int, default=8, help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--learning_rate", default=3e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.01, type=float, help="Weight decay if we apply some.")
    # parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument("--num_train_epochs", default=10, type=float, help="Total number of training epochs to perform.")
    parser.add_argument("--max_steps", default=-1, type=int, help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
    parser.add_argument("--warmup_proportion", default=0.1, type=float, help="Linear warmup over warmup_proportion.")

    # parser.add_argument("--logging_steps", type=int, default=100, help="Log every X updates steps.")
    # parser.add_argument("--save_steps", type=int, default=500, help="Save checkpoint every X updates steps.")
    parser.add_argument(
        "--eval_all_checkpoints", default=True, action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number",
    )
    parser.add_argument("--no_cuda", default=False, action="store_true", help="Avoid using CUDA when available")
    # parser.add_argument(
    #     "--overwrite_output_dir", default = True, action="store_true", help="Overwrite the content of the output directory",
    # )
    # parser.add_argument(
    #     "--overwrite_cache", default = True, action="store_true", help="Overwrite the cached training and evaluation sets",
    # )
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")

    # parser.add_argument(
    #     "--fp16", action="store_true", help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    # )
    # parser.add_argument(
    #     "--fp16_opt_level", type=str, default="O1",
    #     help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
    #     "See details at https://nvidia.github.io/apex/amp.html",
    # )
    parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
    # parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    # parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")

    parser.add_argument('-f')

args = parser.parse_args()
vars(args)

{'data_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/dataset/ViMMRC_RACE_v1',
 'model_type': 'bert',
 'model_name_or_path': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli',
 'task_name': 'vimmrc_race',
 'output_predictions': True,
 'output_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli_base/epoch_8',
 'freeze_embeddings': False,
 'freeze_layers': None,
 'tb_log_dir': '',
 'config_name': '',
 'tokenizer_name': '',
 'cache_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/cached',
 'max_seq_length': 512,
 'do_train': False,
 'do_eval': False,
 'do_test': True,
 'evaluate_during_training': True,
 'do_lower_case': False,
 'per_gpu_train_batch_size': 4,
 'per_gpu_eval_batch_size': 4,
 'gradient_accumulation_steps': 8,
 'learning_rate': 3e-05,
 'weight_decay': 0.01,
 'max_grad_norm': 1.0,
 'num_train_epochs': 10,
 'max_steps': -1,
 'warmup_proportion': 0.1,
 'eval_all_checkpoints': True,
 'no_cuda': False,
 'seed': 42,
 'local_rank': -1,
 'f': '/root/.loc

In [18]:
if __name__ == "__main__":
    main(args)

INFO:__main__:device: cuda, n_gpu: 1, distributed training False
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at /content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Training/evaluation parameters Namespace(data_dir='/content/drive/MyDrive/NCKH/ViMMRC_model/dataset/ViMMRC_RACE_v1', model_type='bert', model_name_or_path='/content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli', task_name='vimmrc_race', output_predictions=True, output_dir='/content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli_base/epoch_8', freeze_embeddings=F

Evaluating:   0%|          | 0/129 [00:00<?, ?it/s]

INFO:__main__:***** Eval results for vimmrc_race on test for epoch_8 *****
INFO:__main__:  eval_acc = 0.7178988326848249
INFO:__main__:  eval_loss = 0.7739456604494778
INFO:__main__:

INFO:__main__:Saved epoch_8/predictions_true_eval_results.json
INFO:__main__:Saved epoch_8/predictions_true_eval_results_label.json


### dev

In [26]:
parser = argparse.ArgumentParser()

if True:
    # Required parameters
    parser.add_argument(
        "--data_dir", default="{}/dataset/ViMMRC_RACE_v1".format(PRJ_DIR), type=str,
        help="The input data dir. Should contain the .tsv files (or other data files) for the task.",
    )

    parser.add_argument(
        "--model_type", default="bert", type=str,
        help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--model_name_or_path", default="{}/models/mbert_nli_base".format(PRJ_DIR), type=str,
        help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(ALL_MODELS),
    )

    parser.add_argument(
        "--task_name", default="vimmrc_race", type=str,
        help="The name of the task to train selected in the list: " + ", ".join(processors.keys()),
    )
    # parser.add_argument(
    #     "--para_type", default="per_choice", type=str,
    #     choices=["per_choice", "concat_choices", "ignore"],
    #     help="Paragraph building strategy for ARC (default: %(default)s)",
    # )
    parser.add_argument(
        "--output_predictions", default=True, type=bool, help="Whether to export the predictions from the eval step.",
    )
    parser.add_argument(
        "--output_dir", default=PRJ_DIR + '/models/mbert_nli_base'+'/epoch_8', type=str, help="The output directory where the model predictions and checkpoints will be written.",
    )
    parser.add_argument("--freeze_embeddings", default=False, action="store_true", help="Whether to freeze the embeeding layer.",)
    parser.add_argument("--freeze_layers", nargs="*", help="Whether to freeze the embeeding layer.",)

    # Other parameters
    parser.add_argument("--tb_log_dir", default="", type=str, help="Tensorboard log dir for the current experiment")
    parser.add_argument("--config_name", default="", type=str, help="Pretrained config name or path if not the same as model_name")
    parser.add_argument("--tokenizer_name", default="", type=str, help="Pretrained tokenizer name or path if not the same as model_name")
    parser.add_argument(
        "--cache_dir", default="{}/models/cached".format(PRJ_DIR), type=str, help="Where do you want to store the pre-trained models downloaded from s3",
    )
    parser.add_argument(
        "--max_seq_length", default=MAX_SEQ_LENGTHS['vimmrc'], type=int,
        help="The maximum total input sequence length after tokenization. Sequences longer than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument("--do_train", default = False, action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", default = True, action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument("--do_test", default = False, action="store_true", help="Whether to run test on the test set")
    parser.add_argument(
        "--evaluate_during_training", action="store_true", default = True, help="Run evaluation during training at each logging step.",
    )
    parser.add_argument(
        "--do_lower_case", action="store_true", default = False, help="Set this flag if you are using an uncased model.",
    )

    parser.add_argument("--per_gpu_train_batch_size", default=4, type=int, help="Batch size per GPU/CPU for training.",)
    parser.add_argument("--per_gpu_eval_batch_size", default=4, type=int, help="Batch size per GPU/CPU for evaluation.",)
    parser.add_argument(
        "--gradient_accumulation_steps", type=int, default=8, help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--learning_rate", default=3e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.01, type=float, help="Weight decay if we apply some.")
    # parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument("--num_train_epochs", default=10, type=float, help="Total number of training epochs to perform.")
    parser.add_argument("--max_steps", default=-1, type=int, help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
    parser.add_argument("--warmup_proportion", default=0.1, type=float, help="Linear warmup over warmup_proportion.")

    # parser.add_argument("--logging_steps", type=int, default=100, help="Log every X updates steps.")
    # parser.add_argument("--save_steps", type=int, default=500, help="Save checkpoint every X updates steps.")
    parser.add_argument(
        "--eval_all_checkpoints", default=True, action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number",
    )
    parser.add_argument("--no_cuda", default=False, action="store_true", help="Avoid using CUDA when available")
    # parser.add_argument(
    #     "--overwrite_output_dir", default = True, action="store_true", help="Overwrite the content of the output directory",
    # )
    # parser.add_argument(
    #     "--overwrite_cache", default = True, action="store_true", help="Overwrite the cached training and evaluation sets",
    # )
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")

    # parser.add_argument(
    #     "--fp16", action="store_true", help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    # )
    # parser.add_argument(
    #     "--fp16_opt_level", type=str, default="O1",
    #     help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
    #     "See details at https://nvidia.github.io/apex/amp.html",
    # )
    parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
    # parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    # parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")

    parser.add_argument('-f')

args = parser.parse_args()
vars(args)

{'data_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/dataset/ViMMRC_RACE_v1',
 'model_type': 'bert',
 'model_name_or_path': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli_base',
 'task_name': 'vimmrc_race',
 'output_predictions': True,
 'output_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli_base/epoch_8',
 'freeze_embeddings': False,
 'freeze_layers': None,
 'tb_log_dir': '',
 'config_name': '',
 'tokenizer_name': '',
 'cache_dir': '/content/drive/MyDrive/NCKH/ViMMRC_model/models/cached',
 'max_seq_length': 512,
 'do_train': False,
 'do_eval': True,
 'do_test': False,
 'evaluate_during_training': True,
 'do_lower_case': False,
 'per_gpu_train_batch_size': 4,
 'per_gpu_eval_batch_size': 4,
 'gradient_accumulation_steps': 8,
 'learning_rate': 3e-05,
 'weight_decay': 0.01,
 'max_grad_norm': 1.0,
 'num_train_epochs': 10,
 'max_steps': -1,
 'warmup_proportion': 0.1,
 'eval_all_checkpoints': True,
 'no_cuda': False,
 'seed': 42,
 'local_rank': -1,
 'f': '/root

In [27]:
if __name__ == "__main__":
    main(args)

INFO:__main__:device: cuda, n_gpu: 1, distributed training False
INFO:__main__:Training/evaluation parameters Namespace(data_dir='/content/drive/MyDrive/NCKH/ViMMRC_model/dataset/ViMMRC_RACE_v1', model_type='bert', model_name_or_path='/content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli_base', task_name='vimmrc_race', output_predictions=True, output_dir='/content/drive/MyDrive/NCKH/ViMMRC_model/models/mbert_nli_base/epoch_8', freeze_embeddings=False, freeze_layers=None, tb_log_dir='', config_name='', tokenizer_name='', cache_dir='/content/drive/MyDrive/NCKH/ViMMRC_model/models/cached', max_seq_length=512, do_train=False, do_eval=True, do_test=False, evaluate_during_training=True, do_lower_case=False, per_gpu_train_batch_size=4, per_gpu_eval_batch_size=4, gradient_accumulation_steps=8, learning_rate=3e-05, weight_decay=0.01, max_grad_norm=1.0, num_train_epochs=10, max_steps=-1, warmup_proportion=0.1, eval_all_checkpoints=True, no_cuda=False, seed=42, local_rank=-1, f='/root/.local/

Evaluating:   0%|          | 0/74 [00:00<?, ?it/s]

INFO:__main__:***** Eval results for vimmrc_race on dev for epoch_8 *****
INFO:__main__:  eval_acc = 0.7857142857142857
INFO:__main__:  eval_loss = 0.6062577972062737
INFO:__main__:

INFO:__main__:Saved epoch_8/predictions_false_eval_results.json
INFO:__main__:Saved epoch_8/predictions_false_eval_results_label.json
INFO:__main__:Loading features from cached file /content/drive/MyDrive/NCKH/ViMMRC_model/dataset/ViMMRC_RACE_v1/cached_dev_mbert_nli_base_512_vimmrc_race
INFO:__main__:***** Running evaluation for vimmrc_race on dev for mbert_nli_base *****
INFO:__main__:  Num examples = 294
INFO:__main__:  Batch size = 4


Evaluating:   0%|          | 0/74 [00:00<?, ?it/s]

INFO:__main__:***** Eval results for vimmrc_race on dev for mbert_nli_base *****
INFO:__main__:  eval_acc = 0.782312925170068
INFO:__main__:  eval_loss = 0.6848980618665952
INFO:__main__:

INFO:__main__:Saved mbert_nli_base/predictions_false_eval_results.json
INFO:__main__:Saved mbert_nli_base/predictions_false_eval_results_label.json
